<a href="https://colab.research.google.com/github/veenqa/Driver-Drowsiness-Detection/blob/main/fatigue_monitory_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python tensorflow  numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
# import tkinter as tk # Removed tkinter as it requires a GUI
# from tkinter import filedialog # Removed filedialog

def load_images_from_folder(folder_path, label, img_size=150):
    images = []
    labels = []
    # Check if the folder path exists and is a directory
    if not os.path.exists(folder_path):
        print(f"Error: Folder not found at {folder_path}")
        return [], []
    if not os.path.isdir(folder_path):
        print(f"Error: {folder_path} is not a directory")
        return [], []

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        # Check if the path is a file before trying to read it
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (img_size, img_size))
                images.append(img)
                labels.append(label)
    return images, labels

def main():
    # Ask user for input paths using input()
    drowsy_path = input("Enter the full path to the Drowsy images folder: ").strip()
    non_drowsy_path = input("Enter the full path to the NonDrowsy images folder: ").strip()

    if not drowsy_path or not non_drowsy_path:
        print("❌ Folder paths not provided. Exiting.")
        return

    IMG_SIZE = 150

    # Load data
    drowsy_images, drowsy_labels = load_images_from_folder(drowsy_path, 1, IMG_SIZE)
    non_drowsy_images, non_drowsy_labels = load_images_from_folder(non_drowsy_path, 0, IMG_SIZE)

    # Check if any images were loaded
    if not drowsy_images and not non_drowsy_images:
        print("❌ No images loaded from the provided paths. Please check the paths and try again.")
        return

    # Combine data
    data = np.array(drowsy_images + non_drowsy_images) / 255.0
    labels = np.array(drowsy_labels + non_drowsy_labels)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # CNN model
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train model
    print("\n🚀 Training the model...\n")
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=2)

    # Evaluate model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"\n✅ Test Accuracy: {test_acc * 100:.2f}%")

    # Save model
    model.save("fatigue_model_input.h5") # Changed filename to reflect input method
    print("📦 Model saved as 'fatigue_model_input.h5'")

if __name__ == "__main__":
    main()

Enter the full path to the Drowsy images folder: /content/drive/MyDrive/Drowsy
Enter the full path to the NonDrowsy images folder: /content/drive/MyDrive/Non Drowsy


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



🚀 Training the model...

Epoch 1/10
8/8 - 12s - 1s/step - accuracy: 0.5403 - loss: 0.7027 - val_accuracy: 0.6935 - val_loss: 0.6125
Epoch 2/10
8/8 - 9s - 1s/step - accuracy: 0.6169 - loss: 0.6619 - val_accuracy: 0.6774 - val_loss: 0.6322
Epoch 3/10
8/8 - 9s - 1s/step - accuracy: 0.6573 - loss: 0.6075 - val_accuracy: 0.7097 - val_loss: 0.5658
Epoch 4/10
8/8 - 9s - 1s/step - accuracy: 0.7419 - loss: 0.5291 - val_accuracy: 0.7903 - val_loss: 0.5135
Epoch 5/10
8/8 - 9s - 1s/step - accuracy: 0.7621 - loss: 0.4720 - val_accuracy: 0.7903 - val_loss: 0.4118
Epoch 6/10
8/8 - 9s - 1s/step - accuracy: 0.8024 - loss: 0.3995 - val_accuracy: 0.8065 - val_loss: 0.3845
Epoch 7/10
8/8 - 9s - 1s/step - accuracy: 0.8790 - loss: 0.3244 - val_accuracy: 0.8226 - val_loss: 0.3444
Epoch 8/10
8/8 - 9s - 1s/step - accuracy: 0.8629 - loss: 0.2934 - val_accuracy: 0.8710 - val_loss: 0.3014
Epoch 9/10
8/8 - 9s - 1s/step - accuracy: 0.9032 - loss: 0.2831 - val_accuracy: 0.8871 - val_loss: 0.2189
Epoch 10/10
8/8 - 8


✅ Test Accuracy: 88.71%
📦 Model saved as 'fatigue_model_input.h5'


In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os

# Load model
model = load_model("fatigue_model_input.h5")

# Path to the directory containing images
img_dir_path = "/content/drive/MyDrive/fatiguevalidate"

# Check if the directory exists
if not os.path.isdir(img_dir_path):
    print(f"Error: Directory not found at {img_dir_path}")
else:
    # Iterate through images in the directory
    for filename in os.listdir(img_dir_path):
        img_path = os.path.join(img_dir_path, filename)

        # Check if it's a file and not a directory
        if os.path.isfile(img_path):
            # Load and preprocess image
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (150, 150))
                img = img / 255.0
                img = np.expand_dims(img, axis=0)

                # Predict
                prediction = model.predict(img)

                # Print prediction for each image
                print(f"Prediction for {filename}:")
                if prediction[0][0] > 0.5:
                    print("🔴 Drowsy detected")
                else:
                    print("🟢 Non-Drowsy detected")
            else:
                print(f"Error: Could not read image from {img_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Prediction for ZC1343.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Prediction for ZC1309.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction for ZC1296.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction for ZC1307.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction for ZC1331.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Prediction for ZC1302.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Prediction for ZC1339.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Prediction for ZC1344.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction for ZC1338.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Prediction for ZC1321.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Prediction for ZC1332.png:
🟢 Non-Drowsy detected
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predi

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load model
model = load_model("fatigue_model_input.h5")

# Load and preprocess image
img_path = "/content/drive/MyDrive/A0003.png"
img = cv2.imread(img_path)
img = cv2.resize(img, (150, 150))
img = img / 255.0
img = np.expand_dims(img, axis=0)

# Predict
prediction = model.predict(img)
if prediction[0][0] > 0.5:
    print("🔴 Drowsy detected")
else:
    print("🟢 Non-Drowsy detected")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
🔴 Drowsy detected
